In [1]:
import numpy as np
import damask

In [2]:
result_file = "dwell_tensionX.hdf5"


In [3]:
result = damask.Result(result_file)
last_result = result.view(increments=-1)

In [4]:
last_result

Created by DAMASK_grid v3.0.0-alpha7
        on 2023-05-12 23:03:04+0530
 executing "DAMASK_grid --load tensionX.yaml --geom dwell.vti"

increment_140 (110.0 s)
  phase
    Ti
      mechanical
        F / 1: deformation gradient
        F_e / 1: elastic deformation gradient
        F_p / 1: plastic deformation gradient
        IPFcolor_(0 1 0) / 8-bit RGB: Inverse Pole Figure (IPF) colors along sample direction (0 1 0)
        IPFcolor_(1 0 0) / 8-bit RGB: Inverse Pole Figure (IPF) colors along sample direction (1 0 0)
        L_p / 1/s: plastic velocity gradient
        O / q_0 (q_1 q_2 q_3): crystal orientation as quaternion
        P / Pa: first Piola-Kirchhoff stress
        epsilon_V^0.0(F) / 1: strain tensor of F (deformation gradient)
        epsilon_V^0.0(F)_vM / 1: Mises equivalent strain of epsilon_V^0.0(F) (strain tensor of F (deformation gradient))
        sigma / Pa: Cauchy stress calculated from P (first Piola-Kirchhoff stress) and F (deformation gradient)
        sigma_v

In [5]:
stress = last_result.get("sigma_vM")

In [6]:
stress

{'Ti': array([1.13476523e+09, 1.15459386e+09, 1.15043883e+09, ...,
        1.15708192e+09, 1.14404998e+09, 1.16112006e+09]),
 'beta': array([1.20370482e+09, 8.41017485e+08, 7.91837255e+08])}

In [7]:
strain = last_result.get('epsilon_V^0.0(F)_vM')
strain

{'Ti': array([0.07684245, 0.11431583, 0.1038925 , ..., 0.09496835, 0.12189584,
        0.12914259]),
 'beta': array([0.17228658, 0.14685873, 0.19834386])}

In [8]:
texture = last_result.get("O")

In [9]:
texture

{'Ti': array([[ 0.00743491, -0.96589537,  0.035268  ,  0.25641181],
        [ 0.16997271,  0.9591459 ,  0.01217056, -0.22583246],
        [ 0.47499827, -0.28267436, -0.04348183, -0.83221462],
        ...,
        [ 0.21557   ,  0.29821294, -0.39158019,  0.84336444],
        [ 0.64860537, -0.20428867,  0.58896083,  0.43669481],
        [ 0.65178534, -0.21599327,  0.59144043,  0.42275405]]),
 'beta': array([[ 0.0256851 ,  0.53215105,  0.1144743 , -0.83848147],
        [ 0.25165161,  0.68770956, -0.42051258, -0.53562691],
        [ 0.32175739, -0.40121774, -0.67050516, -0.53471426]])}

In [10]:
texture_alpha_quat = damask.Orientation.from_quaternion(q=texture['Ti'], lattice="hP")
texture_alpha_euler = texture_alpha_quat.as_Euler_angles()
texture_alpha_euler

array([[4.64690406, 2.62275353, 4.71989818],
       [5.37027194, 2.56848137, 5.34489539],
       [2.24206816, 0.58009776, 1.93681469],
       ...,
       [0.40061138, 1.02924263, 2.24048329],
       [2.49724268, 1.34612848, 4.97108061],
       [2.49634348, 1.36219653, 4.93763578]])

In [11]:
texture_beta_quat = damask.Orientation.from_quaternion(q=texture['beta'], lattice="cI")
texture_beta_euler = texture_beta_quat.as_Euler_angles()
texture_beta_euler

array([[4.95489949, 1.15116719, 4.53112507],
       [4.60279818, 1.87501786, 5.700417  ],
       [3.14406001, 1.79374439, 1.08092419]])

In [12]:
import h5py
import pandas as pd
with h5py.File("dwell_tensionX.hdf5", "r") as file:
    position_data = file['cell_to']['phase'][()]

label_and_position = [[str(data[0][0])[2:-1], int(data[0][1])]for data in position_data]
df = pd.DataFrame({
    "abs_position": [i for i in range(len(label_and_position))],
    "phase": [data[0] for data in label_and_position],
    "position": [data[1] for data in label_and_position],
})
df

,abs_position,phase,position
0,0,Ti,0
1,1,Ti,1
2,2,Ti,2
3,3,Ti,3
4,4,Ti,4
...,...,...,...
12235,12235,Ti,12232
12236,12236,Ti,12233
12237,12237,Ti,12234
12238,12238,Ti,12235


In [13]:
all_euler_texture = []
all_stress = []
all_strain = []
for i in range(len(df)):
#     print(df.iloc[i, :])
    if str(df.iat[i, 1]) == "Ti":
        index = int(df.iat[i, 2])
        all_euler_texture.append(list(texture_alpha_euler[index]))
        all_stress.append(stress['Ti'][index])
        all_strain.append(strain["Ti"][index])
    if str(df.iat[i, 1]) == "beta":
        index = int(df.iat[i, 2])
        all_euler_texture.append(list(texture_beta_euler[index]))
        all_stress.append(stress['beta'][index])
        all_strain.append(strain['beta'][index])
    
all_euler_texture

[[4.646904062107777, 2.6227535344644712, 4.719898180547601],
 [5.37027193741369, 2.5684813717455897, 5.344895385594658],
 [2.242068159922831, 0.5800977601697038, 1.936814689942346],
 [4.838643344641569, 2.712225838244144, 4.820204554094372],
 [5.318769832704926, 1.6986076130381838, 1.6853958956763766],
 [1.7515148986630273, 0.44378192040985287, 1.3571146188866683],
 [4.767285914550616, 2.650358856834389, 4.917537583648485],
 [4.732925807302126, 2.6544412580989003, 4.992034572240275],
 [4.665576081695932, 2.7064596838491957, 1.7900727302745345],
 [1.5085014799823202, 0.4224329015878488, 1.3631036368258205],
 [4.677205553969621, 2.7562278241079707, 1.7545628198998784],
 [1.5331227762836417, 0.40439204461814093, 1.4486756276090718],
 [1.4683421043582507, 0.41594159422350657, 1.4994038943874655],
 [1.4598977194737863, 0.41884763016318927, 1.485190944013866],
 [4.709409575964904, 2.7529968858671845, 4.839430998401292],
 [5.159160569185378, 2.6426210548935085, 5.358082144511527],
 [5.1677565

In [14]:
df.query("phase == 'beta'")

,abs_position,phase,position
1781,1781,beta,0
11282,11282,beta,1
11818,11818,beta,2


In [15]:
all_euler_texture[1781]

[4.954899487332725, 1.1511671902578735, 4.5311250690460305]

In [16]:
all_stress[1781]

1203704815.8250792

In [17]:
original_data_file = "Dwell_tension_Initial.ang"
header_lines_in_original = 180
export_file_name = "PostDeformedTexture.ang"
IQ_index = 5
CI_index = 6

In [18]:
header = []
data_lines = []
with open(original_data_file, "r") as file:
    all_data = file.readlines()
    for i in range(header_lines_in_original):
        header.append(all_data[i])
    for i in range(header_lines_in_original, len(all_data)):
        data_lines.append(all_data[i].split())
print(header)

['# HEADER: Start\n', '# TEM_PIXperUM          1.000000\n', '# x-star                0.486910\n', '# y-star                0.579235\n', '# z-star                0.583326\n', '# WorkingDistance       18.040001\n', '# SampleTiltAngle       69.900002\n', '# CameraElevationAngle  10.000000\n', '# CameraAzimuthalAngle  0.000000\n', '#\n', '# Phase 2\n', '# MaterialName  \tTi (Titanium, beta)\n', '# Formula     \tTi\n', '# Info\t\t\n', '# Symmetry              43\n', '# PointGroupID          131\n', '# LatticeConstants      3.310 3.310 3.310  90.000  90.000  90.000\n', '# NumberFamilies        100\n', '# hklFamilies   \t 0 -1  1 1 6.689197 1\n', '# hklFamilies   \t 0 -2  0 1 4.647529 1\n', '# hklFamilies   \t 1 -2 -1 1 3.676836 1\n', '# hklFamilies   \t 0 -2  2 0 3.063804 0\n', '# hklFamilies   \t 0 -3  1 1 2.628516 1\n', '# hklFamilies   \t 2 -2 -2 0 2.308226 0\n', '# hklFamilies   \t 1 -3 -2 0 2.050676 0\n', '# hklFamilies   \t 0 -4  0 0 1.840713 0\n', '# hklFamilies   \t 0 -3  3 0 1.67357

In [19]:
# Updating data lines
for i, data in enumerate(data_lines):
    for j in range(3):
        data[j] = str(round(all_euler_texture[i][j], 6))
    data[IQ_index] = str(round(all_strain[i], 6))
    data[CI_index] = str(round(all_stress[i], 3))
data_lines

[['4.646904',
  '2.622754',
  '4.719898',
  '0.00000',
  '0.00000',
  '0.076842',
  '1134765231.841',
  '1',
  '11873',
  '0.510',
  '630.688721',
  '2081.770996',
  '164.769699'],
 ['5.370272',
  '2.568481',
  '5.344895',
  '8.00000',
  '0.00000',
  '0.114316',
  '1154593864.978',
  '1',
  '11849',
  '1.842',
  '634.964111',
  '2138.382080',
  '165.466400'],
 ['2.242068',
  '0.580098',
  '1.936815',
  '16.00000',
  '0.00000',
  '0.103892',
  '1150438827.628',
  '1',
  '13917',
  '1.216',
  '638.482727',
  '2040.389038',
  '161.172501'],
 ['4.838643',
  '2.712226',
  '4.820205',
  '24.00000',
  '0.00000',
  '0.095586',
  '1164447914.439',
  '1',
  '10587',
  '0.926',
  '609.376221',
  '2350.924072',
  '171.288193'],
 ['5.31877',
  '1.698608',
  '1.685396',
  '32.00000',
  '0.00000',
  '0.09357',
  '1303148601.948',
  '1',
  '11132',
  '1.463',
  '634.567078',
  '2240.319092',
  '170.920105'],
 ['1.751515',
  '0.443782',
  '1.357115',
  '40.00000',
  '0.00000',
  '0.1056',
  '1191303133

In [20]:
# Save the data as ang
with open(export_file_name, "w") as write_file:
    for line in header:
        write_file.write(line)
    for data in data_lines:
        line = ""
        for datas in data:
            line += datas
#             print(datas)
            line += "\t"
#         print(line)
        write_file.write(line + "\n")